# Bonkers Clean Data 2015

The purpose of this file/notebook is to clean/preprocess the marathon dataset so that it can be used to machine learning and other analysis.

## Clean the Data
Clean the data so that it can preprocessed easily.

    1. Import raw csv file.
    2. Drop the columns we are not interested in.
    3. Convert columns to the appropriate data types.
    4. Drop any rows with missing data.
    5. Save the cleaned data as a csv file.

In [1]:
# import dependencies
import pandas as pd
from datetime import datetime as dt

In [2]:
# read in csv files

file_path = "Resources/"
marathon_2015_df = pd.read_csv(f"{file_path}marathon_results_2015.csv")
marathon_2015_df

,Unnamed: 0,Bib,Name,Age,M/F,City,State,Country,Citizen,Unnamed: 9,...,25K,30K,35K,40K,Pace,Proj Time,Official Time,Overall,Gender,Division
0,0,3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:02:39,0:04:56,-,2:09:17,1,1,1
1,1,4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,NaN,NaN,...,1:16:07,1:31:59,1:47:59,2:02:42,0:04:58,-,2:09:48,2,2,2
2,2,8,"Chebet, Wilson",29,M,Marakwet,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:03:01,0:04:59,-,2:10:22,3,3,3
3,3,11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:48:03,2:03:47,0:05:00,-,2:10:47,4,4,4
4,4,10,"Korir, Wesley",32,M,Kitale,NaN,KEN,NaN,NaN,...,1:16:07,1:32:00,1:47:59,2:03:27,0:05:00,-,2:10:49,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26593,26593,25656,"Prescott, Francine J.",64,F,Boynton Beach,FL,USA,NaN,MI,...,4:25:53,5:19:44,6:17:19,7:13:34,0:17:31,-,7:38:56,26594,12015,269
26594,26594,31359,"Emerson, Annette C.",61,F,Old Town,ME,USA,NaN,NaN,...,4:32:44,5:31:58,6:28:56,7:26:19,0:17:59,-,7:51:30,26595,12016,270
26595,26595,25559,"Cerveny, Nona",66,F,Scottsdale,AZ,USA,NaN,NaN,...,4:33:51,5:38:56,6:38:51,7:36:18,0:18:18,-,7:59:33,26596,12017,91
26596,26596,29396,"Buccilli, Alfred P.",53,M,Lynnfield,MA,USA,NaN,NaN,...,4:29:20,5:31:11,6:33:35,7:35:38,0:18:20,-,8:00:37,26597,14580,2055


In [3]:
# function to convert columns from string to datetime

def to_time(df_col):
    # for each item in the column, append yymmdd, to string, then convert to datetime
    time_df_col=[]
    yymmdd='04/20/15'

    for item in df_col:
        # add the yymmdd to the hhmmss string
        full_date=f"{yymmdd} {item}"
        
        # try to convert string to datetime, if theres an error, replace item with "-"
        try:
            # convert to datetime, then append to the time_df_col
            time_df_col.append(dt.strptime(full_date, '%m/%d/%y %H:%M:%S'))
        except ValueError:
            #time_df_col.append("error")
            time_df_col.append("-")
            
    # return the converted column
    return time_df_col

In [4]:
def to_minutes(df_col):
    df_col_minutes=[]
    # for each item in the column
    for item in df_col:
        # extract the hour, minute, second
        h, m, s = item.split(":")
        
        # convert to int
        h=int(h)
        m=int(m)
        s=int(s)
        
        # calculate the minutes, round to 2 decimal places
        minutes=round(m+h*60+s/60, 2)
        
        # append to new col
        df_col_minutes.append(minutes)

    return df_col_minutes

In [5]:
def to_int(df_col):
    df_col_int=[]
    
    # for each value in the column
    for item in df_col:
        #convert to int
        item=int(item)
        
        #append to new list
        df_col_int.append(item)
    # return the new list
    return df_col_int

In [6]:
def clean_data(marathon_df):
    # look at all column labels/names
    marathon_df.columns
    
    # remove any unnecessary columns
    try:
        marathon_df=marathon_df.drop(labels=["Unnamed: 0", "Citizen", "Unnamed: 9", "Half", "Proj Time", "Overall", "Gender", "Division"], axis=1)
    except KeyError: 
        marathon_df=marathon_df.drop(labels=["Citizen", "Unnamed: 8", "Half", "Proj Time", "Overall", "Gender", "Division"], axis=1)
    
    # set the index as bib number, then drop bib column
    marathon_df=marathon_df.set_index("Bib", drop=True)

    # check dataframe data types
    marathon_df.dtypes
    
    # convert 5K, 10K, 15K, 20K, 25K, 30K, 35K, 40K, Official Time columns to datetime
    # The unit for these columns is just time hh:mm:ss
    
    marathon_df["5K"]=to_time(marathon_df["5K"])
    marathon_df["10K"]=to_time(marathon_df["10K"])
    marathon_df["15K"]=to_time(marathon_df["15K"])
    marathon_df["20K"]=to_time(marathon_df["20K"])
    marathon_df["25K"]=to_time(marathon_df["25K"])
    marathon_df["30K"]=to_time(marathon_df["30K"])
    marathon_df["35K"]=to_time(marathon_df["35K"])
    marathon_df["40K"]=to_time(marathon_df["40K"])
    marathon_df["Official Time"]=to_time(marathon_df["Official Time"])
    
    # convert Pace column from mm:ss to minutes only, rounded to 2 decimal places
    # unit is minutes/Kilometer, for overall race
    marathon_df["Pace"]=to_minutes(marathon_df["Pace"])
    
    # Remove rows where there is a "-" value

    for index in marathon_df.index:

        # check each relavent column. if value is -, then delete entire row.

        if marathon_df.loc[index, "5K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "10K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "15K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "20K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "25K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "30K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "35K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "40K"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "Pace"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
        elif marathon_df.loc[index, "Official Time"] == "-":
            marathon_df=marathon_df.drop(index, axis=0)
    

    return marathon_df

In [7]:
# using the functions above, clean the dataset
marathon_2015_clean = clean_data(marathon_2015_df)
marathon_2015_clean

,Name,Age,M/F,City,State,Country,5K,10K,15K,20K,25K,30K,35K,40K,Pace,Official Time
Bib,,,,,,,,,,,,,,,,
3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,2015-04-20 00:14:43,2015-04-20 00:29:43,2015-04-20 00:44:57,2015-04-20 01:00:29,2015-04-20 01:16:07,2015-04-20 01:32:00,2015-04-20 01:47:59,2015-04-20 02:02:39,4.93,2015-04-20 02:09:17
4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,2015-04-20 00:14:43,2015-04-20 00:29:43,2015-04-20 00:44:58,2015-04-20 01:00:28,2015-04-20 01:16:07,2015-04-20 01:31:59,2015-04-20 01:47:59,2015-04-20 02:02:42,4.97,2015-04-20 02:09:48
8,"Chebet, Wilson",29,M,Marakwet,NaN,KEN,2015-04-20 00:14:43,2015-04-20 00:29:43,2015-04-20 00:44:57,2015-04-20 01:00:29,2015-04-20 01:16:07,2015-04-20 01:32:00,2015-04-20 01:47:59,2015-04-20 02:03:01,4.98,2015-04-20 02:10:22
11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,2015-04-20 00:14:43,2015-04-20 00:29:44,2015-04-20 00:45:01,2015-04-20 01:00:29,2015-04-20 01:16:07,2015-04-20 01:32:00,2015-04-20 01:48:03,2015-04-20 02:03:47,5.00,2015-04-20 02:10:47
10,"Korir, Wesley",32,M,Kitale,NaN,KEN,2015-04-20 00:14:43,2015-04-20 00:29:44,2015-04-20 00:44:58,2015-04-20 01:00:28,2015-04-20 01:16:07,2015-04-20 01:32:00,2015-04-20 01:47:59,2015-04-20 02:03:27,5.00,2015-04-20 02:10:49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25656,"Prescott, Francine J.",64,F,Boynton Beach,FL,USA,2015-04-20 00:50:15,2015-04-20 01:43:31,2015-04-20 02:36:53,2015-04-20 03:32:26,2015-04-20 04:25:53,2015-04-20 05:19:44,2015-04-20 06:17:19,2015-04-20 07:13:34,17.52,2015-04-20 07:38:56
31359,"Emerson, Annette C.",61,F,Old Town,ME,USA,2015-04-20 00:48:36,2015-04-20 01:39:39,2015-04-20 02:39:13,2015-04-20 03:35:58,2015-04-20 04:32:44,2015-04-20 05:31:58,2015-04-20 06:28:56,2015-04-20 07:26:19,17.98,2015-04-20 07:51:30
25559,"Cerveny, Nona",66,F,Scottsdale,AZ,USA,2015-04-20 00:53:03,2015-04-20 01:47:16,2015-04-20 02:41:45,2015-04-20 03:37:07,2015-04-20 04:33:51,2015-04-20 05:38:56,2015-04-20 06:38:51,2015-04-20 07:36:18,18.30,2015-04-20 07:59:33


In [14]:
# export the cleaned dataset as a new csv
output_path="Output_Files/"
marathon_2015_clean.to_csv(f"{output_path}marathon_2015_results_cleaned.csv")

## Preprocess the Data
Using the cleaned datasets, preprocess the data by creating new calculated columns to prepare for machine learning and analysis.

    1. Calculate the amount of minutes it takes for runners to get from one milestone to the next.
    2. Calculate the pace of each runner by taking minutes/distance.
    3. Add a column for the Pace between each milestone.
    4. Drop distance columns.
    5. Export the dataframe as a csv file.

In [9]:
# function to calculate the minute it takes for runners to go from one milestone to the next
def calc_minute(df_col_final, df_col_initial="-"):
    
    # list to hold results
    minutes_list = []
    
    # for each row
    for row_index in range(len(df_col_final)):
        # find the difference between the final and initial columns
        if type(df_col_initial) == str:
            diff=df_col_final[row_index]
            
            # extract seconds, minutes, hours
            s=diff.second
            m=diff.minute
            h=diff.hour

            # calculate total minutes, round to 2 decimals
            minute = m+h*60+s/60
        else:
            diff=df_col_final[row_index]-df_col_initial[row_index]
            
            # extract total seconds
            s = diff.total_seconds()

            # convert to minutes, round to 2 decimal places
            minute = round(s/60, 2)
        
        # append to list
        minutes_list.append(minute)
            
    # return the list
    return minutes_list

In [10]:
# function to calculate the pace from one point to another
def calc_pace(df, milestone_final, milestone_initial=0):
    # calculate the miles traveled
    miles=milestone_final-milestone_initial
    
    # calculate the minutes it takes to travel from one milestone to the next
    time_col_final=df[f"{milestone_final}K"]
    if milestone_initial == 0:
        minutes=calc_minute(time_col_final)
    else:
        time_col_initial=df[f"{milestone_initial}K"]
        minutes=calc_minute(time_col_final, time_col_initial)
    
    # calculate the pace for each row in minutes list, then append to pace_list
    pace_list=[]
    for row in minutes:
        pace=round(row/miles, 2)
        pace_list.append(pace)
        
    return pace_list

In [11]:
# function to add columns for paces between milestones
def pace_column(df):
    df["Pace (0-5K)"]=calc_pace(df, 5)
    df["Pace (5-10K)"]=calc_pace(df, 10, 5)
    df["Pace (10-15K)"]=calc_pace(df, 15, 10)
    df["Pace (15-20K)"]=calc_pace(df, 20, 15)
    df["Pace (20-25K)"]=calc_pace(df, 25, 20)
    df["Pace (25-30K)"]=calc_pace(df, 30, 25)
    df["Pace (30-35K)"]=calc_pace(df, 35, 30)
    df["Pace (35-40K)"]=calc_pace(df, 40, 35)

    # delete time columns and rearrange paces
    df=df.drop(["5K", "10K", "15K", "20K", "25K", "30K", "35K", "40K", "Official Time"], axis=1)
    df=df[["Name", "Age", "M/F", "City", "State", "Country", 
          "Pace (0-5K)", "Pace (5-10K)", "Pace (10-15K)", "Pace (15-20K)", 
          "Pace (20-25K)", "Pace (25-30K)", "Pace (30-35K)", "Pace (35-40K)",
         "Pace"]]
    df=df.rename(columns={"Pace":"Overall Pace"})
    
    return df

In [12]:
marathon_2015_paces=pace_column(marathon_2015_clean)
marathon_2015_paces

,Name,Age,M/F,City,State,Country,Pace (0-5K),Pace (5-10K),Pace (10-15K),Pace (15-20K),Pace (20-25K),Pace (25-30K),Pace (30-35K),Pace (35-40K),Overall Pace
Bib,,,,,,,,,,,,,,,
3,"Desisa, Lelisa",25,M,Ambo,NaN,ETH,2.94,3.00,3.05,3.11,3.13,3.18,3.20,2.93,4.93
4,"Tsegay, Yemane Adhane",30,M,Addis Ababa,NaN,ETH,2.94,3.00,3.05,3.10,3.13,3.17,3.20,2.94,4.97
8,"Chebet, Wilson",29,M,Marakwet,NaN,KEN,2.94,3.00,3.05,3.11,3.13,3.18,3.20,3.01,4.98
11,"Kipyego, Bernard",28,M,Eldoret,NaN,KEN,2.94,3.00,3.06,3.09,3.13,3.18,3.21,3.15,5.00
10,"Korir, Wesley",32,M,Kitale,NaN,KEN,2.94,3.00,3.05,3.10,3.13,3.18,3.20,3.09,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25656,"Prescott, Francine J.",64,F,Boynton Beach,FL,USA,10.05,10.65,10.67,11.11,10.69,10.77,11.52,11.25,17.52
31359,"Emerson, Annette C.",61,F,Old Town,ME,USA,9.72,10.21,11.91,11.35,11.35,11.85,11.39,11.48,17.98
25559,"Cerveny, Nona",66,F,Scottsdale,AZ,USA,10.61,10.84,10.90,11.07,11.35,13.02,11.98,11.49,18.30


In [ ]:
# export data to csv
marathon_2015_paces.to_csv()